In [50]:
import yfinance as yf
import pandas as pd
import numpy as np

In [51]:
equities_list = ["amzn", "tsla", "wmt", "meta", "nvda"]
df_equities = pd.DataFrame({
                            "Equity": ["amzn", "tsla", "wmt", "meta", "nvda"],
                            "CumulativeReturns": np.zeros(5),
                            "Volatility (Annualised)": np.zeros(5),
                            "SharpeRatio": np.zeros(5),
                            "SortinoRatio" : np.zeros(5),
                            "MaxDrawDown": np.zeros(5)
                            }).set_index("Equity")

indices_list = ["DJI", "^GSPC", "^IXIC", "^RUT", "^NYA"]
df_indices = pd.DataFrame({
                            "Index": ["DJI", "^GSPC", "^IXIC", "^RUT", "^NYA"],
                            "CumulativeReturns": np.zeros(5),
                            "Volatility (Annualised)": np.zeros(5),
                            "SharpeRatio": np.zeros(5),
                            "SortinoRatio" : np.zeros(5),
                            "MaxDrawDown": np.zeros(5)
                            }).set_index("Index")

In [52]:
def cumulative_returns(closing_price):
    """
    Return the cumulative return of a stock/index.
    """

    daily_returns = closing_price.pct_change()
    cum_returns = (1 + daily_returns).cumprod()
    recent_cum_return = cum_returns[-1]

    return recent_cum_return

def volatility(closing_price):
    """
    Return the volatilty of a stock/index.
    """

    daily_returns = closing_price.pct_change()
    volty = daily_returns.std()
    annualised_volatility = volty * np.sqrt(252)

    return annualised_volatility

def sharpe_ratio(closing_price,risk_free_rate = 0):
    """
    Return the Sharpe Ratio of a stock/index
    """

    daily_returns = closing_price.pct_change()
    excess_returns = daily_returns - risk_free_rate/252
    annualised_volatility = daily_returns.std() * np.sqrt(252)

    sortino = excess_returns.mean() / annualised_volatility

    return sortino

def sortino_ratio(closing_price,risk_free_rate = 0):
    """
    Return the Sortino Ratio of a stock/index
    """

    daily_returns = closing_price.pct_change()
    excess_returns = daily_returns - risk_free_rate/252
    downside_deviation = excess_returns[excess_returns < 0].std()

    sortino = excess_returns.mean() / downside_deviation

    return sortino

def max_drawdown(closing_price):
    """
    Return the Maximum DrawDown of a stock/index
    """

    daily_returns = closing_price.pct_change()
    cum_returns = (1 + daily_returns).cumprod()
    max_possible_cum_return = cum_returns.cummax()
    daily_drawdown = (cum_returns - max_possible_cum_return) / max_possible_cum_return

    mdd = daily_drawdown.min()

    return mdd

In [53]:
for equity in equities_list:
    hist = yf.download(equity, start="2010-01-01")['Adj Close']

    df_equities.loc[equity,"CumulativeReturns"] = cumulative_returns(hist)
    df_equities.loc[equity,"Volatility (Annualised)"] = volatility(hist)
    df_equities.loc[equity,"SharpeRatio"] = sharpe_ratio(hist,0.02)
    df_equities.loc[equity,"SortinoRatio"] = sortino_ratio(hist,0.02)
    df_equities.loc[equity,"MaxDrawDown"] = max_drawdown(hist)

for index in indices_list:
    ticker = yf.Ticker(index)
    hist = ticker.history(start="2010-01-01")["Close"]

    df_indices.loc[index,"CumulativeReturns"] = cumulative_returns(hist)
    df_indices.loc[index,"Volatility (Annualised)"] = volatility(hist)
    df_indices.loc[index,"SharpeRatio"] = sharpe_ratio(hist,0.02)
    df_indices.loc[index,"SortinoRatio"] = sortino_ratio(hist,0.02)
    df_indices.loc[index,"MaxDrawDown"] = max_drawdown(hist)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [54]:
print(df_equities)
print("\n-------------------------------------------------------------\n")
print(df_indices)

        CumulativeReturns  Volatility (Annualised)  SharpeRatio  SortinoRatio   
Equity                                                                          
amzn            16.938013                 0.331621     0.002954      0.066950  \
tsla           104.554189                 0.573069     0.003497      0.081255   
wmt              3.728283                 0.194014     0.001992      0.041822   
meta             6.246927                 0.407035     0.002242      0.047355   
nvda            68.872805                 0.447126     0.003519      0.082387   

        MaxDrawDown  
Equity               
amzn      -0.561453  
tsla      -0.736322  
wmt       -0.364351  
meta      -0.767361  
nvda      -0.663351  

-------------------------------------------------------------

       CumulativeReturns  Volatility (Annualised)  SharpeRatio  SortinoRatio   
Index                                                                          
DJI             3.202126                 0.176020     